In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import sys
from sklearn.preprocessing import normalize

In [2]:
xy = np.loadtxt('C:\\Users\\SANHA\\Desktop\\new_data.csv', delimiter=',', dtype=np.float32)
x_data=xy[:,0:-1]
x_data = normalize(x_data, axis=0, norm='max')
xy[:,0:-1]=x_data
y_data=xy[:,[-1]]
m_data=xy[:499,:]
r_data=xy[:10000,:]
g_data=xy[:0,:]
F_data=xy[:,:]

In [3]:
# Model Inputs
def model_inputs(real_dim, noise_dim):
    inputs_real_ = tf.placeholder(tf.float32, shape=[None, real_dim], name='inputs_real')
    inputs_z_ = tf.placeholder(tf.float32, shape=[None, noise_dim], name='inputs_z')
    
    return inputs_real_, inputs_z_

def leaky_relu(x, alpha):
    return tf.maximum(alpha * x, x)

In [4]:
# Generator Network
def model_generator(z_input, out_dim, n_units=128, reuse=False, alpha=0.01):
    # used to reuse variables, name scope
    with tf.variable_scope('generator', reuse=reuse):
        hidden_layer = tf.layers.dense(z_input, n_units, activation=None)
        hidden_layer = leaky_relu(hidden_layer, alpha)
        
        logits = tf.layers.dense(hidden_layer, out_dim, activation=None)
        outputs = tf.nn.sigmoid(logits)
        
        return outputs, logits

In [5]:
# Discriminator Network
def model_discriminator(input, n_units=128, reuse=False, alpha=0.1):
    with tf.variable_scope('discriminator', reuse=reuse):
        hidden_layer = tf.layers.dense(input, n_units, activation=tf.nn.relu)
        #hidden_layer = leaky_relu(hidden_layer, alpha)
        
        logits = tf.layers.dense(hidden_layer, 1, activation=None)
        outputs = tf.nn.sigmoid(logits)
        
        return outputs, logits

In [6]:
#parameter
input_size = 42
z_dim = 21
g_hidden_size = 128
d_hidden_size = 128
alpha = 0.1
smooth = 0.1
learning_rate = 0.001

In [7]:
tf.reset_default_graph()  # If we don't have this, as we call this block over and over, the graph gets bigger and bigger

graph = tf.Graph()
with graph.as_default():
    inputs_real, inputs_z = model_inputs(input_size, z_dim)
    
    g_outputs, g_logits = model_generator(inputs_z, input_size, n_units=g_hidden_size, reuse=False, alpha=alpha)
    
    d_outputs_real, d_logits_real = model_discriminator(inputs_real, n_units=d_hidden_size, reuse=False, alpha=alpha)
    d_outputs_fake, d_logits_fake = model_discriminator(g_outputs, n_units=d_hidden_size, reuse=True, alpha=alpha)
    
    d_loss_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=d_logits_real, labels=tf.ones_like(d_logits_real) * (1-smooth)))
    d_loss_fake = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=d_logits_fake, labels=tf.zeros_like(d_logits_fake)))
    
    d_loss = d_loss_real + d_loss_fake
    g_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=d_logits_fake, labels=tf.ones_like(d_logits_fake)))
    
    t_vars = tf.trainable_variables()
    g_vars = [variable for variable in t_vars if 'generator' in variable.name]
    d_vars = [variable for variable in t_vars if 'discriminator' in variable.name]
    
    # Affected Variables with var_list
    d_optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(d_loss, var_list=d_vars)
    g_optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(g_loss, var_list=g_vars)
    
    # Saving variables with var_list
    saver = tf.train.Saver(var_list=g_vars)

In [11]:
samples=[]
normal=0
abnormal=0

with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    f = open('C:\\Users\\SANHA\\Desktop\\gen_sample3333.txt', 'a+')
    for step in range(1):
        
        
        batch_images = F_data[step].reshape([1, 42])
        batch_z = np.random.uniform(-1, 1, size=[1, z_dim])
        
        _ = sess.run(d_optimizer, feed_dict={inputs_real : batch_images, inputs_z : batch_z})
        _ = sess.run(g_optimizer, feed_dict={inputs_z : batch_z})
        loss_d, loss_g = sess.run([d_loss, g_loss], feed_dict={inputs_real : batch_images, inputs_z : batch_z})
        #if step%1000==0:
        #    print('step {} / {} Complete. D_Loss : {:0.3f}, G_Loss : {:0.3f}'.format(step+1, 100000, loss_d, loss_g))
        sys.stdout.write("\r%d / %d: %f, %f" % (step, 100000, loss_d, loss_g))
        sys.stdout.flush()
        sample_z = np.random.uniform(-1, 1, size=[1, z_dim])  # 16 Samples each epoch
        gen_samples, _ = sess.run(model_generator(inputs_z, input_size, reuse=True), feed_dict={inputs_z : sample_z})
        
        temp=gen_samples[0,41]
        #print(temp)
        if temp>=0.5:
            gen_samples[0,41]=1
            abnormal+=1
        else :
            gen_samples[0,41]=0
            normal+=1
        #print(temp,gen_samples[0,41])
        #write for text to csv
        f.write("%f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f \n" %(gen_samples[0,0],gen_samples[0,1],gen_samples[0,2],gen_samples[0,3],gen_samples[0,4],gen_samples[0,5],gen_samples[0,6],gen_samples[0,7],gen_samples[0,8],gen_samples[0,9],gen_samples[0,10],gen_samples[0,11],gen_samples[0,12],gen_samples[0,13],gen_samples[0,14],gen_samples[0,15],gen_samples[0,16],gen_samples[0,17],gen_samples[0,18],gen_samples[0,19],gen_samples[0,20],gen_samples[0,21],gen_samples[0,22],gen_samples[0,23],gen_samples[0,24],gen_samples[0,25],gen_samples[0,26],gen_samples[0,27],gen_samples[0,28],gen_samples[0,29],gen_samples[0,30],gen_samples[0,31],gen_samples[0,32],gen_samples[0,33],gen_samples[0,34],gen_samples[0,35],gen_samples[0,36],gen_samples[0,37],gen_samples[0,38],gen_samples[0,39],gen_samples[0,40],gen_samples[0,41]))
       
        m_data=np.append(m_data,gen_samples,axis=0)
        g_data=np.append(g_data,gen_samples,axis=0)
       
        
    print('Generating Complete. normal={}, abnormal={}'.format(normal,abnormal))
    f.close()

0 / 100000: 1.054104, 1.193582Generating Complete. normal=1, abnormal=0


In [18]:
xy2 = np.loadtxt('C:\\Users\\SANHA\\Desktop\\gen_data.csv', delimiter=',', dtype=np.float32)
gx_data=xy2[:,0:-1]
gy_data=xy2[:,[-1]]
gF_data=xy2[:,:]


x1_data=m_data[:,0:-1]
y1_data=m_data[:,[-1]]
x2_data=r_data[:,0:-1]
y2_data=r_data[:,[-1]]


x3_data=g_data[:,0:-1]
y3_data=g_data[:,[-1]]


nb_classes=2

X=tf.placeholder(tf.float32,[None,41])
Y=tf.placeholder(tf.int32,[None,1])

Y_one_hot=tf.one_hot(Y,nb_classes)
Y_one_hot=tf.reshape(Y_one_hot,[-1,nb_classes])

W1=tf.Variable(tf.random_normal([41,82]),name='weight1')
b1=tf.Variable(tf.random_normal([82]),name='bias1')
layer1=tf.sigmoid(tf.matmul(X,W1)+b1)

W2=tf.Variable(tf.random_normal([82,42]),name='weight2')
b2=tf.Variable(tf.random_normal([42]),name='bias2')
layer2=tf.sigmoid(tf.matmul(layer1,W2)+b2)

W3=tf.Variable(tf.random_normal([42,nb_classes]),name='weight3')
b3=tf.Variable(tf.random_normal([nb_classes]),name='bias3')
logits=tf.matmul(layer2,W3)+b3
hypothesis=tf.nn.softmax(logits)

cost_i=tf.nn.softmax_cross_entropy_with_logits(logits=logits,labels=Y_one_hot)

cost=tf.reduce_mean(cost_i)
optimizer=tf.train.AdamOptimizer(learning_rate=0.01).minimize(cost)

prediction=tf.argmax(hypothesis,1) #가능성을 퍼센트로~~
correct_prediction=tf.equal(prediction,tf.arg_max(Y_one_hot,1))
accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for step in range(1000):
        sess.run(optimizer,feed_dict={X:x_data,Y:y_data})
        #print("training by gan sample")
        if step %10==0:
            loss,acc=sess.run([cost,accuracy],feed_dict={X:gx_data,Y:gy_data})
            print("step: {:5}\tLoss: {:.3f}\tAcc: {:.2%}".format(step,loss,acc))
  
    tr=0
    fa=0
    total=0


    pred = sess.run(prediction, feed_dict={X: gx_data})
    for p, y in zip(pred, gy_data.flatten()):
            if(p==int(y)):
                tr=tr+1
            else:
                fa=fa+1
            #print("[{}] Prediction: {} Real Y: {}".format(p == int(y), p, int(y)))

    print("true={} false: {} acc: {:0.2f}".format(tr,fa,tr/(tr+fa)))

step:     0	Loss: 0.851	Acc: 52.35%
step:    10	Loss: 0.233	Acc: 90.19%
step:    20	Loss: 0.215	Acc: 91.28%
step:    30	Loss: 0.215	Acc: 91.31%
step:    40	Loss: 0.217	Acc: 91.40%
step:    50	Loss: 0.211	Acc: 91.77%
step:    60	Loss: 0.217	Acc: 91.82%
step:    70	Loss: 0.221	Acc: 91.88%
step:    80	Loss: 0.227	Acc: 91.85%
step:    90	Loss: 0.230	Acc: 91.87%
step:   100	Loss: 0.234	Acc: 91.81%
step:   110	Loss: 0.238	Acc: 91.77%
step:   120	Loss: 0.242	Acc: 91.78%
step:   130	Loss: 0.247	Acc: 91.80%
step:   140	Loss: 0.252	Acc: 91.79%
step:   150	Loss: 0.257	Acc: 91.72%
step:   160	Loss: 0.263	Acc: 91.67%
step:   170	Loss: 0.269	Acc: 91.54%
step:   180	Loss: 0.275	Acc: 91.35%
step:   190	Loss: 0.282	Acc: 91.24%
step:   200	Loss: 0.288	Acc: 91.09%
step:   210	Loss: 0.295	Acc: 90.96%
step:   220	Loss: 0.302	Acc: 90.88%
step:   230	Loss: 0.308	Acc: 90.85%
step:   240	Loss: 0.315	Acc: 90.73%
step:   250	Loss: 0.323	Acc: 90.59%
step:   260	Loss: 0.330	Acc: 90.48%
step:   270	Loss: 0.338	Acc: